In [1]:
import os
from pathlib import Path
from urllib.parse import quote_plus
import psycopg2
import pandas as pd

In [2]:
# === config: читаем .env / окружение ===
try:
    from dotenv import load_dotenv, find_dotenv  # pip install python-dotenv
    load_dotenv(find_dotenv())
    
except Exception:
    pass

PGHOST = os.getenv("PGHOST", "localhost")
PGPORT = int(os.getenv("PGPORT", "5432"))
PGUSER = os.getenv("PGUSER", "postgres")
PGPASSWORD = os.getenv("PGPASSWORD", "postgres")
PGDATABASE = os.getenv("PGDATABASE", "startml_mock")

def get_conn_params():
    return dict(
        host=PGHOST,
        port=PGPORT,
        user=PGUSER,
        password=PGPASSWORD,
        database=PGDATABASE,
    )

def get_db_url():
    # экранируем пароль на всякий
    return f"postgresql://{PGUSER}:{quote_plus(PGPASSWORD)}@{PGHOST}:{PGPORT}/{PGDATABASE}"

In [3]:
# Вариант 1: через psycopg2
def connect_with_psycopg2(query: str):
    with psycopg2.connect(**get_conn_params()) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            return cur.fetchall()

# Вариант 2: через pandas
def connect_with_pandas(query: str) -> pd.DataFrame:
    return pd.read_sql(query, con=get_db_url())

# SELECT (1/2)

In [4]:
query = """SELECT * FROM sample_user LIMIT 10"""

# Запускаем оба варианта
print("Через psycopg2:")
result = connect_with_psycopg2(query)
display(result)

print("Через pandas:")
result = connect_with_pandas(query)
display(result)

Через psycopg2:


[(1, 0, 35, 'Russia', 'Moscow', 4, 'Android', 'ads'),
 (2, 1, 21, 'Russia', 'Yalutorovsk', 2, 'iOS', 'ads'),
 (3, 0, 45, 'Russia', 'Severodvinsk', 0, 'Android', 'ads'),
 (4, 1, 38, 'Russia', 'Vetluga', 3, 'Android', 'organic'),
 (5, 0, 22, 'Russia', 'Moscow', 4, 'Android', 'ads'),
 (6, 1, 23, 'Russia', 'Slyudyanka', 0, 'iOS', 'organic'),
 (7, 0, 29, 'Russia', 'Moscow', 1, 'iOS', 'organic'),
 (8, 1, 61, 'Russia', 'Irkutsk', 1, 'iOS', 'organic'),
 (9, 1, 39, 'Russia', 'Omsk', 1, 'Android', 'organic'),
 (10, 1, 21, 'Russia', 'Moscow', 1, 'iOS', 'ads')]

Через pandas:


,user_id,gender,age,country,city,exp_group,os,source
0,1,0,35,Russia,Moscow,4,Android,ads
1,2,1,21,Russia,Yalutorovsk,2,iOS,ads
2,3,0,45,Russia,Severodvinsk,0,Android,ads
3,4,1,38,Russia,Vetluga,3,Android,organic
4,5,0,22,Russia,Moscow,4,Android,ads
5,6,1,23,Russia,Slyudyanka,0,iOS,organic
6,7,0,29,Russia,Moscow,1,iOS,organic
7,8,1,61,Russia,Irkutsk,1,iOS,organic
8,9,1,39,Russia,Omsk,1,Android,organic
9,10,1,21,Russia,Moscow,1,iOS,ads


# SELECT (2/2)

In [5]:
query = """
SELECT DISTINCT(topic)
FROM sample_post
"""

# Запускаем оба варианта
print("Через psycopg2:")
result = connect_with_psycopg2(query)
display(result)

print("Через pandas:")
result = connect_with_pandas(query)
display(result)

Через psycopg2:


[('tech',),
 ('covid',),
 ('movie',),
 ('entertainment',),
 ('politics',),
 ('sport',),
 ('business',)]

Через pandas:


,topic
0,tech
1,covid
2,movie
3,entertainment
4,politics
5,sport
6,business


# Фильтры (1/2)

In [6]:
query = """
SELECT *
FROM sample_user
WHERE age > 30 and os = 'iOS'
"""

print("Через pandas:")
result = connect_with_pandas(query)
display(result)

Через pandas:


,user_id,gender,age,country,city,exp_group,os,source
0,8,1,61,Russia,Irkutsk,1,iOS,organic
1,26,0,34,Russia,Petropavlovsk-Kamchatskiy,4,iOS,organic
2,28,0,44,Russia,Yurga,3,iOS,ads
3,33,1,52,Russia,Novorossiysk,1,iOS,ads
4,37,0,38,Russia,Aksay,1,iOS,organic
...,...,...,...,...,...,...,...,...
181,1592,0,36,Russia,Moscow,0,iOS,ads
182,1600,1,34,Russia,Murmansk,0,iOS,ads
183,1601,1,32,Russia,Saint Petersburg,2,iOS,organic
184,1607,0,43,Russia,Moscow,0,iOS,ads


# Фильтры (2/2)

In [7]:
query = """
SELECT *
FROM sample_user
WHERE country != 'Russia' AND ((exp_group NOT IN (0, 3)) OR city = 'Minsk')
LIMIT 10
"""

print("Через pandas:")
result = connect_with_pandas(query)
display(result)

Через pandas:


,user_id,gender,age,country,city,exp_group,os,source
0,16,0,18,Azerbaijan,Sumqayıt,4,Android,organic
1,23,0,28,Azerbaijan,M.Ə. Rəsulzadə,4,Android,ads
2,34,1,46,Ukraine,Nemyriv,2,Android,ads
3,42,0,21,Belarus,Asipovichy,1,Android,ads
4,45,1,17,Belarus,Polatsk,2,iOS,ads
5,63,1,45,Turkey,Istanbul,4,Android,ads
6,67,1,54,Ukraine,Kyiv,2,Android,organic
7,83,0,26,Kazakhstan,Esik,4,Android,ads
8,84,1,33,Ukraine,Zolotonosha,2,Android,organic
9,95,1,22,Ukraine,Kherson,2,Android,organic


# Группировка (1/6)

In [8]:
query = """
SELECT country, AVG(age)
FROM sample_user
GROUP BY country
"""

print("Через pandas:")
result = connect_with_pandas(query)
display(result)

Через pandas:


,country,avg
0,Finland,28.357143
1,Belarus,24.193548
2,Estonia,19.000000
3,Cyprus,25.000000
4,Turkey,26.347826
5,Ukraine,30.220779
6,Kazakhstan,27.909091
7,Latvia,15.000000
8,Azerbaijan,28.705882
9,Russia,27.367729


# Группировка (2/6)

In [9]:
query = """
SELECT exp_group, os, COUNT(user_id) AS "total_users", MAX(age) AS "max_age", MIN(age) AS "min_age"
FROM sample_user
GROUP BY exp_group, os
"""

print("Через pandas:")
result = connect_with_pandas(query)
display(result)

Через pandas:


,exp_group,os,total_users,max_age,min_age
0,4,iOS,113,61,14
1,0,Android,208,64,14
2,0,iOS,124,72,14
3,3,iOS,102,52,14
4,2,iOS,144,52,14
5,3,Android,208,57,14
6,1,Android,212,62,14
7,2,Android,210,77,14
8,1,iOS,95,61,14
9,4,Android,216,63,14


# Группировка (3/6)

In [10]:
query = """
SELECT topic
FROM sample_post
WHERE LENGTH("text") > 5000
GROUP BY topic
"""

print("Через pandas:")
result = connect_with_pandas(query)
display(result)

Через pandas:


,topic
0,movie
1,entertainment
2,business
3,tech


# Группировка (4/6)

In [11]:
query = """
SELECT country, COUNT('id')
FROM sample_user
GROUP BY country
HAVING COUNT('id') > 1000
"""

print("Через pandas:")
result = connect_with_pandas(query)
display(result)

Через pandas:


,country,count
0,Russia,1444


# Группировка (5/6)

In [12]:
query = """
SELECT country, exp_group, COUNT(user_id)
FROM sample_user
GROUP BY country, exp_group;
"""

print("Через pandas:")
result = connect_with_pandas(query)
display(result)

Через pandas:


,country,exp_group,count
0,Turkey,0,7
1,Finland,3,1
2,Turkey,3,2
3,Belarus,2,13
4,Azerbaijan,0,2
5,Finland,2,4
6,Finland,0,1
7,Belarus,1,5
8,Russia,1,275
9,Kazakhstan,3,7


# Группировка (6/6)

In [13]:
query = """
SELECT exp_group, COUNT(user_id)
FROM sample_user
WHERE city = 'Moscow'
GROUP BY exp_group
HAVING AVG(age) > 27.2
"""

print("Через pandas:")
result = connect_with_pandas(query)
display(result)

Через pandas:


,exp_group,count
0,1,32
1,3,47


# Сортировка (1/2)

In [14]:
query = """
SELECT topic, COUNT(post_id) as post_count
FROM sample_post
GROUP BY topic
ORDER BY post_count DESC
"""

print("Через pandas:")
result = connect_with_pandas(query)
display(result)

Через pandas:


,topic,post_count
0,movie,140
1,covid,103
2,business,35
3,sport,23
4,entertainment,18
5,politics,18
6,tech,14


# Сортировка (2/2)

In [15]:
query = """
SELECT *
FROM sample_user
WHERE city = 'Voronezh'
ORDER BY age DESC, exp_group ASC
"""

print("Через pandas:")
result = connect_with_pandas(query)
display(result)

Через pandas:


,user_id,gender,age,country,city,exp_group,os,source
0,1252,0,51,Russia,Voronezh,0,Android,ads
1,545,1,45,Russia,Voronezh,2,Android,organic
2,1120,1,38,Russia,Voronezh,2,iOS,ads
3,222,1,37,Russia,Voronezh,3,iOS,ads
4,1277,1,23,Russia,Voronezh,1,Android,ads
5,953,1,21,Russia,Voronezh,1,Android,ads
6,1289,1,21,Russia,Voronezh,1,Android,ads
7,1443,0,21,Russia,Voronezh,2,Android,organic
8,319,0,21,Russia,Voronezh,3,iOS,ads


# Join (1/2)

In [16]:
query = """
SELECT
    fa.post_id,
    fa.timestamp as time,
    u.age,
    u.os
FROM
    sample_feed_action fa
JOIN
    sample_user u ON fa.user_id = u.user_id
WHERE
    u.city = 'Omsk'
    AND fa.action = 'like'
ORDER BY
    fa.timestamp DESC
LIMIT 100
"""

print("Через pandas:")
result = connect_with_pandas(query)
display(result)

# Сохраняем в CSV файл
# result.to_csv('omsk_likes.csv', index=False, sep=',')

Через pandas:


,post_id,time,age,os
0,350,2021-12-28 06:36:36,16,iOS
1,116,2021-12-23 16:52:36,16,iOS
2,195,2021-10-22 08:13:33,19,Android


# Join (2/2)

In [17]:
query = """
SELECT
    u.city
FROM
    sample_feed_action fa
JOIN
    sample_user u ON fa.user_id = u.user_id
JOIN
    sample_post p ON fa.post_id = p.post_id
WHERE
    u.age = 39  -- u.age = 36
    AND fa.action = 'view'
    AND DATE(fa.timestamp) = '2021-12-01'
    AND p.topic = 'covid'
GROUP BY
    u.city
ORDER BY
    COUNT(fa.action) ASC
LIMIT 1
"""

print("Через pandas:")
result = connect_with_pandas(query)
display(result)

Через pandas:


,city
0,Bogotol


# Топ постов

In [18]:
query = """
SELECT
    fa.post_id,
    COUNT(DISTINCT fa.user_id),
    MAX(fa.timestamp)
FROM
    sample_feed_action fa 
WHERE
    fa.action = 'like'
GROUP BY
    fa.post_id
ORDER BY
    COUNT(DISTINCT fa.user_id) DESC 
LIMIT 3;
"""

print("Через pandas:")
result = connect_with_pandas(query)
display(result)

Через pandas:


,post_id,count,max
0,304,11,2021-12-12 09:55:32
1,188,8,2021-12-18 17:06:31
2,301,8,2021-12-17 06:43:54
